Install dependencies

geopy python API fetches latitude and longitude of a given place. Here I have used US city names for generating dataset from the uploaded .txt file

In [0]:
!pip install geopy

The modin pandas is faster compared to pandas. It parallizes dataframe/table computation on the 4 cores of laptop. For mode details on modin: https://pypi.org/project/modin/


In [0]:
# The faster pandas
!pip install modin[all]

# Dataset Creation

Upload the text file containing ciy names

In [24]:
from google.colab import files
uploaded = files.upload()

Saving Street_Names.csv to Street_Names.csv


In [25]:
!ls

city_names.txt	sample_data  Street_Names.csv


**Parse the city names from the file**

In [0]:
city_names=[]       # List of city names
with open('city_names.txt') as f:
  for line in f.readlines():
    #line = line.replace('\n',',')
    city_names.append(line)
city_names=city_names[2:]       #First two elements are discarded as the link and a new line character came along in txt file parsing

*The following stores the corresponding latitudes and longitudes for the city names as there will be in the SQL database*

In [0]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Radius")
latitude=[]
longitude=[]
for i in city_names:
  location = geolocator.geocode(i)
  latitude.append(location.latitude)
  longitude.append(location.longitude)

Now, the prices, Number of bedrooms, bathrooms and ID are randomly generated from a uniform statistical distribution with specified range. 

In [41]:
import random
id_=[]
price=[]
bedrooms=[]
bathrooms=[]
for i in range(0,len(city_names)):
  id_.append(random.randrange(100, 1000, 2))      # Generate random ID numbers from 100 to 1000 with steps of 2/even
  bedrooms.append(random.randrange(1, 7, 1))      # Generate random # of bedrooms from range of 1 to 7 with even steps of 1.
  bathrooms.append(random.randrange(1, 7, 1))     # Generate random # of bathrooms from range of 1 to 7 with even steps of 1.
  price.append(random.uniform(100000, 1000000))   # Generate random prices from range of 100000 to 1000000 (float)
print(len(id_),len(city_names),len(price),len(bathrooms),len(bedrooms),bathrooms[0:5],bedrooms[0:5])

385 385 385 385 385 [5, 5, 6, 1, 6] [1, 6, 2, 4, 6]


In [47]:
import modin.pandas as pd     #The faster pandas
df=pd.DataFrame({'ID':id_,'latitude':latitude,'longitude':longitude,'price':price,'bathrooms':bathrooms,'bedrooms':bedrooms})  #create table/dataframe
df.head()

,ID,latitude,longitude,price,bathrooms,bedrooms
0,958,57.148243,-2.092809,168617.208681,5,1
1,736,32.446450,-99.747591,157204.642849,5,6
2,326,41.083064,-81.518485,313887.342156,6,2
3,350,42.651167,-73.754968,825370.667999,1,4
4,844,35.084103,-106.650985,734646.686748,6,6


_______________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
**End of dataset creation**
________________________________________________________________________________


**Inputs**: (city name, budget (min, max), # bedrooms (min, max) , # bathrooms (min, max)
For either one input cases (budget, # bedrooms, # bathrooms), one input is taken say min, then max is upper limit of integer and if max is given as input then lower limit of integer is taken as minimum. 
<br>
**Assumption made here**: The either inpput which has not been provided is considered maximum and minimum system integer values to it's respective max and min

Exeption handling is done for the purpose of atlease one input and coorect input

In [48]:
def input_parameters():
  import sys
  city=input("Please enter input city: ")
  flag_budget=0       # Here flags are used to keep track in the either input case which one has been provided.
  flag_bedroom=0
  flag_bathroom=0
  try:
    min_budget=abs(float(input("Enter min budget: ")))
    flag_budget+=1
  except ValueError:
    min_budget=-sys.maxsize
  try:
    max_budget=abs(float(input("Enter max budget: ")))
  except ValueError:
    if(flag_budget==1): max_budget=sys.maxsize
    else: 
      print("Give at least min or max budget")
      return
  try:
    min_bedroom=abs(float(input("Enter min number of bedroom: ")))
    flag_bedroom+=1
  except ValueError:
    min_bedroom=-sys.maxsize
  try:
    max_bedroom=abs(float(input("Enter maximum number of bedroom: ")))
  except ValueError:
    if(flag_bedroom==1): max_bedroom=sys.maxsize
    else:
      print("Give at least min or max bedroom")
      return
  try:
    min_bathroom=abs(float(input("Enter minimum number of bathroom: ")))
    flag_bathroom+=1
  except ValueError:
    min_bathroom=-sys.maxsize
  try:
    max_bathroom=abs(float(input("Enter maximum number of bathroom: ")))
  except ValueError:
    if(flag_bathroom==1): max_bathroom=sys.maxsize
    else:
      print("Give at least min or max bathroom")
      return
  return city,min_budget,max_budget,min_bathroom,max_bathroom,min_bedroom,max_bedroom

try:
  city,min_price,max_price,min_bathroom,max_bathroom,min_bedroom,max_bedroom=input_parameters()
  print(city,min_price,max_price,min_bathroom,max_bathroom,min_bedroom,max_bedroom)
except TypeError:
  print("Please enter details correctly by running this cell again")


Please enter input city: 5TH AVE NY
Enter min budget: 30000
Enter max budget: 500000
Enter min number of bedroom: 1
Enter maximum number of bedroom: 4
Enter minimum number of bathroom: 1
Enter maximum number of bathroom: 7
5TH AVE NY 30000.0 500000.0 1.0 7.0 1.0 4.0


*The following haversine function (Great circle) is used to calculate distance on the basis of latititude and longitude formula*. Reference for this Wikipedia: https://en.wikipedia.org/wiki/Haversine_formula 

In [0]:
### Function returns distance in miles ###
import numpy as np
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959        #Radius of earth in miles
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

The following distance calculation is done as per numpy array vectorization of the entire column dataframe. Hence it is faster as comapred to each row cross differences. Iterative process is slower than vectorized one.

An extra column of distance is created so that large distances like beyond 10 miles can be rejected. In pandas the comparison of dataframe column is faster as modin can parallelize computation comparison, I had SQL or dataframe join in mind, but I believe the operation would be the operation expensive if we have more than 2-3 Million rows.

In [50]:
%%timeit
import time
start=time.time()
from geopy.distance import geodesic
location=geolocator.geocode(city)
df['distance'] = haversine(location.latitude, location.longitude, df['latitude'].values, df['longitude'].values)      # df is the database table
print("Program run time: "+str(time.time()-start))

Program run time: 1.4293692111968994
Program run time: 1.2828428745269775
Program run time: 1.4420299530029297
Program run time: 1.1335101127624512
1 loop, best of 3: 1.13 s per loop


Matched table is the table after filtering accoring to the criteria:
<br>
*For a property and requirement to be considered a valid match, distance should 
be within 10 miles, the budget is +/- 25%, bedroom and bathroom should be +/- 2*

In [51]:
matched_table=pd.DataFrame(
matched_table=df[(df['distance']<=1000) & (df['price']>float(float(min_price)-float(min_price)*0.25)) & (df['price']>float(float(max_price)+float(max_price)*0.25)) & (df['bathrooms']>int(min_bathroom)-2) & (df['bathrooms']<int(max_bathroom)+2) & (df['bedrooms']>int(min_bedroom)-2) & (df['bedrooms']<int(max_bedroom)+2)]

In [52]:
#10 rows of the filtered table
matched_table.head(10)

,ID,latitude,longitude,price,bathrooms,bedrooms,distance
3,350,42.651167,-73.754968,825370.667999,1,4,127.199816
6,652,40.602206,-75.471279,689077.316586,6,5,81.762247
18,834,33.749099,-84.390185,841701.592868,5,5,753.168453
19,714,39.364285,-74.422935,830190.226499,3,1,103.511881
24,292,39.290882,-76.610759,627891.776035,4,5,176.369739
38,676,42.360253,-71.058291,917544.741925,6,4,183.044343
55,312,41.975887,-91.670405,728441.236768,2,3,921.097093
59,426,35.227087,-80.843127,895469.011870,1,2,538.594440
64,214,39.101454,-84.512460,905691.921026,2,5,572.179249
72,592,39.962260,-83.000707,833319.725389,2,4,480.486087


Here the scenario is like this:
for only one case input of budget +-25% has been considered at the time of creation of matched table state, here say we take a case of minimum budget is given as input, then we have max budget as MAX INT, we just need to consider the -10 % for new matched table case as +10% is already considered. This is after comment Ref1.
<br>
<br>
*If bedroom and bathroom fall between min and max, each will contribute full 20%. **If min or max is not given, match percentage varies according to the value**.* 
<br>
The bold section of this point is not clear to me so I have not implemented this case in the following code cells.
<br>
In this matched table itself new columns of match percentage of corresponding attribute is created for theshold filtering in the following cells



In [58]:
%%timeit
matched_table['distance_match percentage']=(np.where((matched_table['distance']<=2)&(matched_table['distance']>=0) ,0,30))
matched_table['price_match percentage']=(np.where((matched_table['price']>=float(min_price)) & (matched_table['price']<=float(max_price)),0,30))
matched_table['bedroom_match percentage']=(np.where((matched_table['bedrooms']>=float(min_bedroom)) & (matched_table['bedrooms']<=float(max_bedroom)),0,20))
matched_table['bathroom_match percentage']=(np.where((matched_table['bathrooms']>=float(min_bathroom)) & (matched_table['bathrooms']<=float(max_bathroom)),0,20))
### Ref:1 For either min or max 
if(min_price==-sys.maxsize):
  matched_table['price_match percentage']=(np.where(matched_table['price']<=float(max_price)+0.1*float(max_price)),0,30)
if(max_price==sys.maxsize):
  matched_table['price_match percentage']=(np.where(matched_table['price']>=float(min_price)-0.1*float(min_price)),0,30)
#if(min_bathroom==-sys.maxsize):
  #matched_table['bathroom_match percentage']=(np.where(matched_table['bathrooms']<=float(min_price)-0.1*float(min_price)),0,30) ##percentage vary according to value isn't clear

1 loop, best of 3: 492 ms per loop


Overall percentage is the summation of all individual attribute contribution, and 40% overall percentage is considered as threshold.
<br>
*All matches above 40% can only be considered useful.*

In [59]:
%%timeit
matched_table['overall percentage']=matched_table['distance_match percentage']+ matched_table['price_match percentage']+matched_table['bedroom_match percentage']+matched_table['bathroom_match percentage']
final_table=pd.DataFrame()
final_table=matched_table[matched_table['overall percentage']>=40]
final_table=final_table.sort_values(by ='overall percentage' , ascending=False)
print(final_table[['ID','distance','overall percentage']].head(60))
print(final_table.shape)

      ID    distance  overall percentage
302  400  876.166656                  80
359  722  915.786158                  80
105  534  164.453387                  80
286  432  431.217116                  80
231  144    7.980758                  80
332  154  641.986901                  80
275  836  902.313712                  80
82   192  485.903717                  80
73   158  206.079538                  80
64   214  572.179249                  80
6    652   81.762247                  80
24   292  176.369739                  80
215  420  881.802544                  80
18   834  753.168453                  80
114  504  255.096885                  80
270  660  318.480260                  60
296  404  248.754827                  60
256  564  946.736538                  60
233  750   13.546463                  60
280  696   61.590124                  60
295  990  404.620581                  60
3    350  127.199816                  60
297  210  789.286665                  60
299  670  738.41

      ID    distance  overall percentage
302  400  876.166656                  80
359  722  915.786158                  80
105  534  164.453387                  80
286  432  431.217116                  80
231  144    7.980758                  80
332  154  641.986901                  80
275  836  902.313712                  80
82   192  485.903717                  80
73   158  206.079538                  80
64   214  572.179249                  80
6    652   81.762247                  80
24   292  176.369739                  80
215  420  881.802544                  80
18   834  753.168453                  80
114  504  255.096885                  80
270  660  318.480260                  60
296  404  248.754827                  60
256  564  946.736538                  60
233  750   13.546463                  60
280  696   61.590124                  60
295  990  404.620581                  60
3    350  127.199816                  60
297  210  789.286665                  60
299  670  738.41

      ID    distance  overall percentage
302  400  876.166656                  80
359  722  915.786158                  80
105  534  164.453387                  80
286  432  431.217116                  80
231  144    7.980758                  80
332  154  641.986901                  80
275  836  902.313712                  80
82   192  485.903717                  80
73   158  206.079538                  80
64   214  572.179249                  80
6    652   81.762247                  80
24   292  176.369739                  80
215  420  881.802544                  80
18   834  753.168453                  80
114  504  255.096885                  80
270  660  318.480260                  60
296  404  248.754827                  60
256  564  946.736538                  60
233  750   13.546463                  60
280  696   61.590124                  60
295  990  404.620581                  60
3    350  127.199816                  60
297  210  789.286665                  60
299  670  738.41

The above is the final search results table.
PS: If the actual more frequent data is provided more better results as per the objective may be achieved. This data is small and widespread across country cities, If more gathered like for each state, each street, town names or it's latitude and longitude is provided then the objective results could be validated correctly.

---
<br>
In a linux environment this could be benchmarked in terms of speed with larger dataset and flags.mark_flag_as_required could be added which would be required attributes.
<br>
________________________________________________________________________________________________________________________________________________________________

